In [ ]:
%pip install awkward pyarrow matplotlib numpy

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import awkward as ak

### Load the data from file

In [ ]:
f = ak.from_parquet('particles.parquet')
pt, eta, phi, met_ref = f.pt, f.eta, f.phi, f.met

### Reference MET implementation

This implementation is as simple as can be:
- run over all events at once
- split the particle momentum into x & y components
- sum the px and py for the particles in each event
- compute the vector magnitude from the MET x & y components

In [ ]:
def compute_met(pt : ak.Array,
                phi : ak.Array):
    # compute the x and y components of each particle's pT
    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    # sum over the particles to calculate the MET vector (px, py)
    met_x = np.sum(px, axis=1)
    met_y = np.sum(py, axis=1)
    # compute the magnitude of the MET vector
    met = np.sqrt(met_x**2 + met_y**2)
    return met

In [ ]:
# compute the MET reference
met_ref = compute_met(pt, phi)

In [ ]:
# load the MET computation from HLS testbench
# a dummy file of zeros is provided, run this again with your real results!
met_hls = np.genfromtxt('met_hls.csv')

### Plot MET distribution

You can overlay your results onto this as another plot.

In [ ]:
bins_spectrum = np.linspace(0, 1000, 101)
hist_ref, _ = np.histogram(met_ref, bins=bins_spectrum)
hist_hls, _ = np.histogram(met_hls, bins=bins_spectrum)
plt.step(bins_spectrum[:-1], hist_ref, label='MET Python Reference')
plt.step(bins_spectrum[:-1], hist_hls, label='MET HLS Implementation')
plt.semilogy()
plt.legend()
plt.xlabel('MET [GeV]')
plt.ylabel('Counts')
plt.savefig('met_distribution.png')
plt.savefig('met_distribution.pdf')

### Plot MET Residual

Plot the residual between the MET reference and yours. A dummy file is provided where the MET is always 0, yours should look much better!

In [ ]:
met_residual = met_ref - met_hls
max_residual = np.max(np.abs(met_residual))
bins_residual = np.linspace(-max_residual, max_residual, 101)
hist_residual, _ = np.histogram(met_residual, bins_residual)

plt.step(bins_residual[:-1], hist_residual, label='Example MET')
#plt.semilogy()
plt.legend()
plt.xlabel('ΔMET(Python, HLS) [GeV]')
plt.ylabel('Counts')
plt.savefig('met_residual.png')
plt.savefig('met_residual.pdf')

## Test

You can use this tolerance test to check whether your implementation is viable. The targets are:
- absolute MET never more than 10 GeV different than the reference
- relative MET never more than 2% different than the reference

In [ ]:
try:
    np.testing.assert_allclose(met_hls, met_ref, rtol=0.02, atol=10)
    print("All Close")
except AssertionError as AE:
    print(AE)

### More tools

This exercise is all about choosing the right precision for your variables. You want to minimise the resources and latency without compromising the MET precision. Below are some visualizations that will help you to choose the right precision.

In [ ]:
f, axs = plt.subplots(1,2, figsize=(10,5))
bins_pt = np.linspace(0, ak.max(pt)*1.1, 101)
hist_pt, _ = np.histogram(ak.ravel(pt), bins=bins_pt)
axs[0].step(bins_pt[:-1], hist_pt)
axs[0].semilogy()
axs[0].set_xlabel("$p_T$ [GeV]")
axs[0].set_ylabel("Count")

bins_phi = np.linspace(-np.pi, np.pi, 101)
hist_phi, _ = np.histogram(ak.ravel(phi), bins=bins_phi)
axs[1].step(bins_phi[:-1], hist_phi)
#axs[0].semilogy()
axs[1].set_xlabel("$\phi$")
axs[1].set_ylabel("Count")
axs[1].set_ylim((0, np.max(hist_phi)*1.1))

In [ ]:
px = pt * np.cos(phi)
py = pt * np.sin(phi)

plt.figure(figsize=(6, 5))
plt.hexbin(ak.ravel(px), ak.ravel(py), gridsize=50, cmap='viridis', norm=matplotlib.colors.LogNorm())
plt.xlabel("$p_x$ [GeV]")
plt.ylabel("$p_y$ [GeV]")
plt.title("Phase Space of $p_x$ vs $p_y$")
plt.colorbar(label="Event count")

In [ ]:
# Try out quantizing different parts of the computation in Python before HLS
# You can use this to help think about the fractional part
# e.g. here we quantize the pt and phi
# consider that the full computation also include quantizing the computations in the compute_met function
pt_q = ak.round(pt / 2**-1) * 2**-1
phi_q = ak.round(phi / 2**-4) * 2**-4

met_q = compute_met(pt_q, phi_q)

print(f"Quantized MET: {met_q[0]:.2f} GeV")
print(f"Reference MET: {met_ref[0]:.2f} GeV")